In [2]:

"""
FILE: skeleton_parser.py
------------------
Author: Firas Abuzaid (fabuzaid@stanford.edu)
Author: Perth Charernwattanagul (puch@stanford.edu)
Modified: 04/21/2014

Skeleton parser for CS564 programming project 1. Has useful imports and
functions for parsing, including:

1) Directory handling -- the parser takes a list of eBay json files
and opens each file inside of a loop. You just need to fill in the rest.
2) Dollar value conversions -- the json files store dollar value amounts in
a string like $3,453.23 -- we provide a function to convert it to a string
like XXXXX.xx.
3) Date/time conversions -- the json files store dates/ times in the form
Mon-DD-YY HH:MM:SS -- we wrote a function (transformDttm) that converts to the
for YYYY-MM-DD HH:MM:SS, which will sort chronologically in SQL.

Your job is to implement the parseJson function, which is invoked on each file by
the main function. We create the initial Python dictionary object of items for
you; the rest is up to you!
Happy parsing!
"""

import sys
from json import loads
from re import sub

columnSeparator = "|"

# Dictionary of months used for date transformation
MONTHS = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06',\
        'Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}

"""
Returns true if a file ends in .json
"""
def isJson(f):
    return len(f) > 5 and f[-5:] == '.json'

"""
Converts month to a number, e.g. 'Dec' to '12'
"""
def transformMonth(mon):
    if mon in MONTHS:
        return MONTHS[mon]
    else:
        return mon

"""
Transforms a timestamp from Mon-DD-YY HH:MM:SS to YYYY-MM-DD HH:MM:SS
"""
def transformDttm(dttm):
    dttm = dttm.strip().split(' ')
    dt = dttm[0].split('-')
    date = '20' + dt[2] + '-'
    date += transformMonth(dt[0]) + '-' + dt[1]
    return date + ' ' + dttm[1]

"""
Transform a dollar value amount from a string like $3,453.23 to XXXXX.xx
"""

def transformDollar(money):
    if money == None or len(money) == 0:
        return money
    return sub(r'[^\d.]', '', money)

"""
Parses a single json file. Currently, there's a loop that iterates over each
item in the data set. Your job is to extend this functionality to create all
of the necessary SQL tables for your database.
"""
def parseJson(json_file):
    with open(json_file, 'r') as f:
        items = loads(f.read())['Items'] # creates a Python dictionary of Items for the supplied json file
        f1 = open("EbayUser.dat", "w")
        f2 = open("Bids.dat", "w")
        f3 = open("Categories.dat", "w")
        f4 = open("Items.dat", "w")
        for item in items:
            """
            TODO: traverse the items dictionary to extract information from the
            given `json_file' and generate the necessary .dat files to generate
            the SQL tables based on your relation design
            """
            
            pass

"""
Loops through each json files provided on the command line and passes each file
to the parser
"""
def main(argv):
    if len(argv) < 2:
        print >> sys.stderr, 'Usage: python skeleton_json_parser.py <path to json files>'
        sys.exit(1)
    # loops over all .json files in the argument
    for f in argv[1:]:
        if isJson(f):
            parseJson(f)
            print("Success parsing " + f)

if __name__ == '__main__':
    main(sys.argv)


KeyError: 'Items'

In [19]:
!%%python TransformationJson.py ebay_data/items-0.json

/bin/sh: line 0: fg: no job control


In [24]:
A = {'a':1,'b':2}
print(A.has_keys('a'))

AttributeError: 'dict' object has no attribute 'has_keys'

In [1]:
%load_ext sql
%sql sqlite:///EbayData.db

'Connected: @EbayData.db'

In [29]:
%%sql
drop table if exists Items;
drop table if exists EbayUsers;
drop table if exists Bids;
drop table if exists Categories;
create table EbayUsers(UserID CHAR(300) PRIMARY KEY,
                      Rating INTEGER NOT NULL,
                      Location CHAR(300),
                      Country CHAR(300));
create table Items(ItemID INTEGER NOT NULL PRIMARY KEY,
                   SellerID CHAR(300) NOT NULL,
                   Name CHAR(300) NOT NULL,
                   Buy_Price DECIMAL,
                   First_Bid DECIMAL NOT NULL,
                   Currently DECIMAL NOT NULL,
                   Number_of_Bids INTEGER NOT NULL,
                   Started TIMESTAMP NOT NULL,
                   Ends TIMESTAMP NOT NULL,
                   Description TEXT NOT NULL,
                   FOREIGN KEY (SellerID) REFERENCES EbayUsers(UserID)
                   );
create table Bids(ItemID INTEGER NOT NULL,
                  UserID CHAR(300) NOT NULL,
                  Time TIMESTAMP NOT NULL,
                  Amount DECIMAL NOT NULL,
                  FOREIGN KEY (ItemID) REFERENCES Items(ItemID),
                  FOREIGN KEY (UserID) REFERENCES EbayUsers(UserID),
                  PRIMARY KEY(ItemID, UserID, Amount)
                 );
create table Categories(ItemID INTEGER NOT NULL,
                 Category_Name CHAR(300) NOT NULL,
                 FOREIGN KEY (ItemID) REFERENCES Items(ItemID),
                 PRIMARY KEY(ItemID, Category_Name)
                 );

 * sqlite://
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

Question 1:

In [25]:
%%sql 
SELECT COUNT(*)
FROM EbayUsers

 * sqlite:///EbayData.db
Done.


COUNT(*)
13422


Question 2:

In [24]:
%%sql 
SELECT COUNT(*)
FROM EbayUsers
WHERE Location = "New York";

 * sqlite:///EbayData.db
Done.


COUNT(*)
80


Question 3:

In [26]:
%%sql
SELECT COUNT(*)
FROM(SELECT COUNT(*) as NUM
FROM Categories
GROUP BY Categories.ItemID)
WHERE NUM = 4;

 * sqlite:///EbayData.db
Done.


COUNT(*)
8365


Question 4:

In [27]:
%%sql
SELECT ItemID
FROM Items
WHERE CURRENTLY = 
(SELECT MAX(CURRENTLY)
FROM Items);

 * sqlite:///EbayData.db
Done.


ItemID
1046871451


Question 5:

In [33]:
%%sql 
SELECT COUNT(DISTINCT EbayUsers.UserID)
FROM Items, EbayUsers
WHERE Items.SellerID = EbayUsers.UserID AND EbayUsers.Rating > 1000;

 * sqlite:///EbayData.db
Done.


COUNT(DISTINCT EbayUsers.UserID)
3130


Question 6:

In [35]:
%%sql
SELECT COUNT(DISTINCT Items.SellerID)
FROM Items, Bids
WHERE Items.SellerID = Bids.UserID

 * sqlite:///EbayData.db
Done.


COUNT(DISTINCT Items.SellerID)
6717


Question 7:

In [37]:
%%sql
SELECT COUNT(DISTINCT Categories.Category_Name)
FROM Categories, Bids
WHERE Categories.ItemID = Bids.ItemID
AND Bids.Amount > 100

 * sqlite:///EbayData.db
Done.


COUNT(DISTINCT Categories.Category_Name)
150


In [ ]:
.separator |
.import EbayUsers.dat EbayUsers
update Items set Location = null where Location = "NULL";
update Items set Country = null where Country = "NULL";

.import Items.dat Items
update Items set Buy_Price = null where Buy_Price = "NULL";
update Items set Description = null where Description = "NULL";

.import Bids.dat Bids
 
.import Categories.dat Categories
                                   